In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datatable

In [ ]:
import datatable as dt

# Import training data
train_data = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data.to_pandas()
train_data = train_data.query('date > 85').reset_index(drop = True) 
train_data = train_data.set_index('ts_id')
# Reduce space taken up in memory
train_data = train_data.astype({c: np.float32 for c, t in train_data.dtypes.items() if t == np.float64})
train_labels = train_data[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']]        
train_data = train_data.drop(train_labels.columns, axis=1)
train_labels['action'] = (train_labels['weight'] * train_labels['resp'] > 0).astype('int')
#train_data = train_data.ffill().bfill()
f_mean = train_data.mean()
train_data.fillna(f_mean)

## I have no need for this data yet
#features = dt.fread('../input/jane-street-market-prediction/features.csv')
#features = features.to_pandas()
#example_test = dt.fread('../input/jane-street-market-prediction/example_test.csv')
#example_test = example_test.to_pandas()
#sample_submission = dt.fread('../input/jane-street-market-prediction/example_sample_submission.csv')
#sample_submission = sample_submission.to_pandas()

train_data.head()

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://www.kaggle.com/marketneutral/purged-rolling-time-series-cv-split
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]


In [ ]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights=w)

In [ ]:
def utility(estimator, X, y):
    """Custom scoring object as per documentation:
    https://scikit-learn.org/stable/modules/model_evaluation.html#implementing-your-own-scoring-object
    Utility score formulae are defined in competition's intro:
    https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation
    Using optimisation tricks from @gogo827jz:
    https://www.kaggle.com/c/jane-street-market-prediction/discussion/201257
    """

    # still looking for a way to write this for xgb.cv but it passes DMatrix which doesn't allow indexing...
    # https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html
    date = train_labels.loc[y.index, 'date'].values
    weight = train_labels.loc[y.index, 'weight'].values
    resp = train_labels.loc[y.index, 'resp'].values
    action = estimator.predict(X)

    p_i = np.bincount(date, weight * resp * action)
    t = p_i.sum() / np.sqrt((p_i ** 2).sum()) * np.sqrt(250 / np.unique(date).size)
    u = np.clip(t, 0, 6) * p_i.sum()
    return u


In [ ]:
import xgboost as xgb
# params = [
#     {'max_depth': 6,
#     'n_estimators':500,},
#     {'max_depth': 11,
#     'n_estimators':500,},
#     {'max_depth': 6,
#     'n_estimators':100,},
#     {'max_depth': 11,
#     'n_estimators':100,},
#     ]

# from sklearn.model_selection import cross_val_score

# cv = PurgedGroupTimeSeriesSplit(n_splits=10, group_gap=31)
# for param in params:
#     clf = xgb.XGBClassifier(
#     **param,
#     missing=None,
#     random_state=1,
#     learning_rate = 0.05,
#     n_jobs = -1,
#     tree_method='gpu_hist',
#     subsample=0.8,
#     colsample_bytree=0.7,
#     sampling_method='gradient_based',
#     #verbosity=2   # info
#     )

#     cv_score = weighted_average(cross_val_score(
#                 estimator=clf,
#                 X=train_data,
#                 y=train_labels['action'],
#                 groups=train_labels['date'],
#                 scoring= utility,
#                 cv=cv,
#             ))
#     print(cv.__class__.__name__, cv_score)
    

In [ ]:
# estimators, depth, subsample, colsample, eval_metric, score
# PurgedGroupTimeSeriesSplit 578.416421985854 500 6
# PurgedGroupTimeSeriesSplit 529.0862793187063 500 11
# PurgedGroupTimeSeriesSplit 377.72291274632397 100 6
# PurgedGroupTimeSeriesSplit 521.1262017084068  100 11


In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier(
    max_depth= 11,
    n_estimators=500,
    missing=None,
    random_state=1,
    learning_rate = 0.05,
    n_jobs = -1,
    tree_method='gpu_hist',
    subsample=0.8,
    colsample_bytree=0.7,
    sampling_method='gradient_based',
    #verbosity=2   # info
    )
clf.fit(train_data,train_labels['action'])

In [ ]:
import pickle
# pickle.dump(clf, open("xgb_gpu.pickle.dat", "wb"))
# clf = pickle.load(open("../input/modelfile/xgb_noise.pickle.dat", "rb"))

In [ ]:
import janestreet
env = janestreet.make_env()

In [ ]:
import time
print("Starting Prediction")
start_time = time.time()
features = train_data.columns.values
rcount = 0
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)
    rcount += len(test_df.index)
print(f"took: {time.time() - start_time} seconds")